<a href="https://colab.research.google.com/github/steffenmodest/Python_Examples/blob/Norm_Weights/M_Net_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

rs = 23  # fester Random Seed

Load Data

In [2]:
from tensorflow.keras.datasets import fashion_mnist, mnist

# Fashion MNIST Daten laden, wir wollen nur die Trainingsdaten und verwerfen die Testdaten
# (X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()
(X_train, y_train), (X_test, y_test) = mnist.load_data()

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

11493376/11490434 [==============================] - 0s 0us/step
(60000, 28, 28)
(10000, 28, 28)
(60000,)
(10000,)


In [3]:
def ifelse(a):
  if a == 0:
    return 1
  else:
    return a

In [4]:
def rms_norm(v_x):
  n_rms = np.sqrt(np.sum(v_x ** 2)/(len(v_x) - 1))
  return v_x/n_rms

Preprocess data

NEW Norm

In [5]:
X = X_train.reshape(len(X_train), -1).astype('int')
X = pd.DataFrame(X)
X = X.applymap(ifelse)
X = X.apply(rms_norm, axis=1)
X = np.array(X)

In [7]:
X

array([[0.01148061, 0.01148061, 0.01148061, ..., 0.01148061, 0.01148061,
        0.01148061],
       [0.01076957, 0.01076957, 0.01076957, ..., 0.01076957, 0.01076957,
        0.01076957],
       [0.01425251, 0.01425251, 0.01425251, ..., 0.01425251, 0.01425251,
        0.01425251],
       ...,
       [0.01299969, 0.01299969, 0.01299969, ..., 0.01299969, 0.01299969,
        0.01299969],
       [0.01400668, 0.01400668, 0.01400668, ..., 0.01400668, 0.01400668,
        0.01400668],
       [0.01375564, 0.01375564, 0.01375564, ..., 0.01375564, 0.01375564,
        0.01375564]])

In [6]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, Normalizer, MinMaxScaler, FunctionTransformer
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Plätte 2D Bild in einen Vektor:
# Reshape behält die Anzahl Element in der ersten Dimension (len(X_orig) -> #Bilder)
# die restlichen Zellen des Arrays (-1 -> Pixel) bekommen alle ihre eigene Dimension
# X = X_train.reshape(len(X_train), -1).astype('float64')

# # Dimensionen um den Mittelpunkt zentrieren
# preproccessing = StandardScaler()
# X = preproccessing.fit_transform(X)
# X = np.array(X)

print ("Originaldaten:")
print("Shape: {}, Mean: {:f}, STD: {:f}".format(X_train.shape, np.mean(X_train), np.std(X_train)))

print ("Vorbereitete Daten:")
print("Shape: {}, Mean: {:f}, STD: {:f}".format(X.shape, np.mean(X), np.std(X)))

Originaldaten:
Shape: (60000, 28, 28), Mean: 33.318421, STD: 78.567490
Vorbereitete Daten:
Shape: (60000, 784), Mean: 0.396041, STD: 0.917538


Create Hidden Activations

In [9]:
H = np.random.rand(60000, 47)
print(H.shape)
print(H)

(60000, 47)
[[0.99149491 0.40398187 0.34400036 ... 0.46452889 0.73011517 0.21260503]
 [0.70847702 0.98409062 0.43405621 ... 0.8279564  0.3379946  0.46118817]
 [0.81164268 0.67531546 0.59596805 ... 0.76526342 0.91809961 0.84069225]
 ...
 [0.03092794 0.29383164 0.68490348 ... 0.30067829 0.89293128 0.45568428]
 [0.24962972 0.2264744  0.56714226 ... 0.14486434 0.17318503 0.953363  ]
 [0.71397981 0.76553312 0.52179234 ... 0.16276917 0.78229085 0.35073355]]


1st Forward Learn

In [10]:
import numpy as np
from sklearn.linear_model import LinearRegression

reg = LinearRegression().fit(H, X)
print(reg.score(H, X))
X_pred = reg.predict(H)
print(f'MAE {mean_absolute_error(X_pred, X)}')
print(f'MSE {mean_squared_error(X_pred, X)}')
print(f'R2 {r2_score(X_pred, X)}')
print(f'R2 vw {r2_score(X_pred, X, multioutput="variance_weighted")}')
print(f'R2 ua {r2_score(X_pred, X, multioutput="uniform_average")}')

W = reg.coef_
print(W.shape)
WI = reg.intercept_
print(WI.shape)

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)


0.000825438237816736
MAE 0.4375399263997524
MSE 0.5934157920761444
R2 -1271.7053861026645
R2 vw -1209.4776783842703
R2 ua -1271.7053861026645
(784, 47)
(784,)


1st Back Activations

In [ ]:
# # X = X_train.reshape(len(X_train), -1).astype('int')
# W = pd.DataFrame(W)
# # X = X.applymap(ifelse)
# W = W.apply(rms_norm)
# W = np.array(X)

In [12]:
XT = X.transpose()
print(f'shape of X_train {X.shape} and X_train transposed {XT.shape}')

reg = LinearRegression().fit(W, XT)
print(reg.score(W, XT))
XT_pred = reg.predict(W)
print(f'MAE {mean_absolute_error(XT_pred, XT)}')
print(f'MSE {mean_squared_error(XT_pred, XT)}')
print(f'R2 {r2_score(XT_pred, XT)}')
print(f'R2 vw {r2_score(XT_pred, XT, multioutput="variance_weighted")}')
print(f'R2 ua {r2_score(XT_pred, XT, multioutput="uniform_average")}')

H_new = reg.coef_
print(H_new.shape)
HI = reg.intercept_
print(HI.shape)

shape of X_train (60000, 784) and X_train transposed (784, 60000)


/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)


0.5503781540173167
MAE 0.45070693403875034
MSE 0.3770613092389211
R2 0.14888888940256373
R2 vw 0.1830674188268436
R2 ua 0.14888888940256373
(60000, 47)
(60000,)


Hidden Norm

In [13]:
# X = H_new_train.reshape(len(H_new_train), -1).astype('int')
H_new = pd.DataFrame(H_new)
# H_new = H_new.applymap(ifelse)
H_new = H_new.apply(rms_norm, axis=1)
H_new = np.array(H_new)

2nd Forward learn - result are Coherent Weights

In [14]:
reg = LinearRegression().fit(H_new, X)
print(reg.score(H_new, X))
X_pred = reg.predict(H_new)
print(f'MAE {mean_absolute_error(X_pred, X)}')
print(f'MSE {mean_squared_error(X_pred, X)}')
print(f'R2 {r2_score(X_pred, X)}')
print(f'R2 vw {r2_score(X_pred, X, multioutput="variance_weighted")}')
print(f'R2 ua {r2_score(X_pred, X, multioutput="uniform_average")}')

W = reg.coef_
print(W.shape)
WI = reg.intercept_
print(WI.shape)

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)


0.7420401249643278
MAE 0.21152485290296427
MSE 0.1532039239451645
R2 -12.498669437080768
R2 vw 0.6523639809261351
R2 ua -12.498669437080768
(784, 47)
(784,)


2nd Back Activation - final representation of X_train in Hidden Activations

In [15]:
reg = LinearRegression().fit(W, XT)
print(reg.score(W, XT))
XT_pred = reg.predict(W)
print(f'MAE {mean_absolute_error(XT_pred, XT)}')
print(f'MSE {mean_squared_error(XT_pred, XT)}')
print(f'R2 vw {r2_score(XT_pred, XT, multioutput="variance_weighted")}')
print(f'R2 ua {r2_score(XT_pred, XT, multioutput="uniform_average")}')

H_new = reg.coef_
print(H_new.shape)
HI = reg.intercept_
print(HI.shape)

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)


0.7854812913253687
MAE 0.29005513499063856
MSE 0.17989941074218724
R2 vw 0.7268952029237173
R2 ua 0.715949996888594
(60000, 47)
(60000,)


In [16]:
# X = H_new_train.reshape(len(H_new_train), -1).astype('int')
H_new = pd.DataFrame(H_new)
# H_new = H_new.applymap(ifelse)
H_new = H_new.apply(rms_norm, axis=1)
H_new = np.array(H_new)

KNN test with Hidden Activations

In [17]:
X = X_test.reshape(len(X_test), -1).astype('int')
X = pd.DataFrame(X)
X = X.applymap(ifelse)
X = X.apply(rms_norm, axis=1)
X = np.array(X)

In [18]:
# X = [[0], [1], [2], [3]]
# y = [0, 0, 1, 1]

# Plätte 2D Bild in einen Vektor:
# Reshape behält die Anzahl Element in der ersten Dimension (len(X_orig) -> #Bilder)
# die restlichen Zellen des Arrays (-1 -> Pixel) bekommen alle ihre eigene Dimension
# X_p_test = X_test.reshape(len(X_test), -1).astype('float64')

# # Dimensionen um den Mittelpunkt zentrieren
# preproccessing = StandardScaler()
# X_p_test = preproccessing.fit_transform(X_p_test)

print ("Originaldaten:")
print("Shape: {}, Mean: {:f}, STD: {:f}".format(X_test.shape, np.mean(X_test), np.std(X_test)))

print ("Vorbereitete Daten:")
print("Shape: {}, Mean: {:f}, STD: {:f}".format(X.shape, np.mean(X), np.std(X)))

from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=5, n_jobs=-1, weights='distance')
neigh.fit(H_new, y_train)

# print(neigh.predict([[1.1]]))

# print(neigh.predict_proba([[0.9]]))

Originaldaten:
Shape: (10000, 28, 28), Mean: 33.791224, STD: 79.172463
Vorbereitete Daten:
Shape: (10000, 784), Mean: 0.398067, STD: 0.916661


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=-1, n_neighbors=5, p=2,
                     weights='distance')

Back Activation of Test Data

In [19]:
XT = X.transpose()
print(f'shape of X_p_train {X.shape} and X_p_train transposed {XT.shape}')

reg = LinearRegression().fit(W, XT)
print(reg.score(W, XT))
XT_pred = reg.predict(W)
print(f'MAE {mean_absolute_error(XT_pred, XT)}')
print(f'MSE {mean_squared_error(XT_pred, XT)}')
print(f'R2 {r2_score(XT_pred, XT)}')
print(f'R2 vw {r2_score(XT_pred, XT, multioutput="variance_weighted")}')
print(f'R2 ua {r2_score(XT_pred, XT, multioutput="uniform_average")}')

H_test = reg.coef_
print(H_test.shape)
HI_test = reg.intercept_
print(HI_test.shape)

shape of X_p_train (10000, 784) and X_p_train transposed (784, 10000)


/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)


0.7912424379251364
MAE 0.2870582230781911
MSE 0.17472950898481382
R2 0.7268773205363537
R2 vw 0.7361648565990903
R2 ua 0.7268773205363537
(10000, 47)
(10000,)


In [20]:
# X = H_new_train.reshape(len(H_new_train), -1).astype('int')
H_test = pd.DataFrame(H_test)
# H_new = H_new.applymap(ifelse)
H_test = H_test.apply(rms_norm, axis=1)
H_test = np.array(H_test)

In [21]:
# Predicting the Test set results
y_pred = neigh.predict(H_test)

In [ ]:
# y_prob = neigh.predict_proba(H_test)

In [ ]:
# print(y_prob[0])

[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]


In [22]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[ 976    1    0    0    0    0    1    1    0    1]
 [   0 1132    2    0    0    0    1    0    0    0]
 [   9    0 1001    3    1    0    1   12    5    0]
 [   0    0    2  980    1    9    0    7    8    3]
 [   2    4    1    0  944    0    5    1    2   23]
 [   4    0    0   12    1  851    9    1    9    5]
 [   8    2    0    0    2    1  944    0    1    0]
 [   2   13   10    1    4    1    0  986    1   10]
 [   7    1    3   10    3    9    5    3  929    4]
 [   6    4    3    5    5    2    1    6    5  972]]


In [23]:
from sklearn.metrics import accuracy_score, classification_report
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.9715
              precision    recall  f1-score   support

           0       0.96      1.00      0.98       980
           1       0.98      1.00      0.99      1135
           2       0.98      0.97      0.97      1032
           3       0.97      0.97      0.97      1010
           4       0.98      0.96      0.97       982
           5       0.97      0.95      0.96       892
           6       0.98      0.99      0.98       958
           7       0.97      0.96      0.96      1028
           8       0.97      0.95      0.96       974
           9       0.95      0.96      0.96      1009

    accuracy                           0.97     10000
   macro avg       0.97      0.97      0.97     10000
weighted avg       0.97      0.97      0.97     10000

